In [ ]:
import pandas as pd
import datetime
import numpy as np
from scipy import stats as st
from scipy.stats import levene
import pylab as pl
from statsmodels.formula.api import ols
import math
#%pip install plotly
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from scipy.stats import ttest_1samp
from scipy.stats import ttest_ind
%pip install pingouin
import pingouin as pg

try:
    games = pd.read_csv('games.csv')
except:
    plan = pd.read_csv('/datasets/games.csv')
    

     ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/35.2 MB 95.8 kB/s eta 0:04:41m

# Task
In this dataset, we are asked to review game platforms based on their sales to say something meaningful about the platform and games market
## Dataset
The dataset includes games on various platforms, some still in existence and others dead.  We have various years of releases of games.  We also have games that appear on different platforms.
## Steps
1- I start by inspecting the data, droping and replacing NaNs as needed, checking for dulicates and change data type as needed.

2- I then consult the distribution of each variable.

3- Based on the above, I slice the dataset to keep only those platforms that have greatest sales and are still in existence.

4- Then, I create a trend chart to examine the evolution of sales by platform over time.

5- I examine the correlation and the causation of reviews on sales.

6- Next, I extract from this dataframe those repeat games to examine their distibtuion by platform over time.

7- I examine the distribution of genre over time and by platform.

8- Then I try to understand the behavior of sales by region.  To do so, I create pie charts and barplots.

9- I follow this by examing the influence of ESRB rating on sales by region.

10- I conduct hypothesis testing to examine rating on platform and genre.

11- I conclude.

# Prepare the data

## Inspect

In [ ]:
games.head()

In [ ]:
games.info()

### Conclusion:
We have 16,715 observations and 13 variables in total.

## Duplicates

In [ ]:
games.duplicated().sum()

### Conclusion:
There are 0 duplicates.

## Lower caps in column names

In [ ]:
games= games.rename(columns=str.lower)

## Missing values

In [ ]:
(games.isnull().sum() / games.shape[0] * 100).round(2)

### Conclusion:
We have 2 missing names, 269 missing years, 2 missing genre, 8578 or 51\% missing critic score, 6701 or 40\% missing user score and 6766 or 40\% missing rating.  Since the percentage of missing values for name and year of release is so slim, I will drop them.  Predict year and name is going to be controversial anyway.

### Dropping some data

In [ ]:
games.dropna(subset=['name'], inplace=True)

In [ ]:
games.dropna(subset=['year_of_release'], inplace=True)

### Change type

In [ ]:
games['year_of_release'] = games['year_of_release'].astype('int')

## Predicting some data

### Critic_score

In [ ]:
games['critic_score'].unique()

In [ ]:
pg.normality(games.critic_score, method='normaltest')

In [ ]:
games.critic_score.describe()

In [ ]:
games['critic_score'].hist(bins = 50);

#### Conclusion:
Though this variable fails the normality test, its median and mean are 68.   I will use the mean to replace nans.

#### Fill the missing cells with mean

In [ ]:
games['critic_score'] = games.groupby(['genre'])['critic_score'].apply(lambda x: x.fillna(x.mean()))

#### Round for data consistency

In [ ]:
games['critic_score'].round()

#### And convert type into integer

In [ ]:
games['critic_score'] = games['critic_score'].astype('int')

### User score

### Convert to numeric

In [ ]:
games['user_score'] = pd.to_numeric(games['user_score'], errors='coerce')

In [ ]:
games['user_score'].unique()

In [ ]:
games.user_score.hist(bins = 50);

In [ ]:
games.user_score.describe()

#### Conclusion:
Though this variable fails the normality test, its median and mean are 7.   I will use the mean to replace nans.

### Replace tbd with nan

In [ ]:
games['user_score'] = games['user_score'].replace(['tbd'], np.nan)

In [ ]:
games['user_score'] = games.groupby(['genre'])['user_score'].apply(lambda x: x.fillna(x.mean()))

### Round to 1 decimal for data consistency

In [ ]:
games['user_score'] = games['user_score'].round(1)

### Rating

#### Strategy
Predicting an ESRB score is a controversial task.  Yet, for the sake of exploration, I will do 2 things here.  For one variable, rating_en, I will predict ESRB using user and critic scores as well as genre.  In the other variable, rating, I will replace the missing values with 0 and encode the rest of the values for future analyses. I will compare the performance of both variables towards the end.

#### Encode to predict rating

In [ ]:
games.rating.value_counts()

In [ ]:
def cond_b(val):
    if val == "E":
        return 1
    if val == "T":
        return 2
    if val == "M":
        return 3
    if val == "E10+":
        return 4
    if val == "EC":
        return 5
    if val == "K-A":
        return 6
    if val == "AO":
        return 7
    if val == "RP":
        return 8  
    
games['rating_na_filled_mode'] = games['rating'].apply(cond_b)

In [ ]:
games['rating_na_filled_mode'] = games.groupby([
    'user_score', 'critic_score', 'genre']).rating_na_filled_mode.transform(
    lambda x: x.fillna(x.mode().iat[0]) if x.notna().any() else x)

In [ ]:
games.rating_na_filled_mode.isna().sum()

In [ ]:
games['rating_na_filled_mode'] = games['rating_na_filled_mode'].fillna(0)

In [ ]:
games['rating_na_filled_mode'].round()

#### Encode to check correlation of rating and sales and replace nas with 0

In [ ]:
def cond_a(val):
    if val == "E":
        return 1
    if val == "T":
        return 2
    if val == "M":
        return 3
    if val == "E10+":
        return 4
    if val == "EC":
        return 5
    if val == "K-A":
        return 6
    if val == "AO":
        return 7
    if val == "RP":
        return 8 
    return 0
    
games['rating'] = games['rating'].apply(cond_a)

#### Change type

In [ ]:
games['rating_na_filled_mode'] = games['rating_na_filled_mode'].astype('int')

## Total Sales

In [ ]:
games['total_sales'] = games[['na_sales','eu_sales','jp_sales','other_sales']].sum(axis=1)

## Check final dataframe

In [ ]:
games.info()

In [ ]:
games.shape

### Conclusion:
The final df has 16444 observations and 13 variables.  We lost around 250 observations.

# Analyze the data

## Games released yearly

In [ ]:
games.groupby(['year_of_release']).agg({'name': 'count'}).plot(
    kind='bar', figsize=(13, 5))
plt.title("Yearly number of games")
plt.legend("")
plt.xlabel('Years')
plt.ylabel('Count');

### Conclusion:
There is a significant difference between years.  Some years have less than 200 games released a year and others have up to 1400.  There is a hike around 2001 in the number of games among the most successful platforms that reaches its peak around 2008 and 2009, but then there is an exponential decrease after 2010.

## Sales difference among platforms

In [ ]:
games.groupby(['platform']).agg({'total_sales': 'sum'}).plot(
    kind='bar', figsize=(13, 5))
plt.title("Total sales by platform")
plt.xlabel('Platform name')
plt.ylabel('Count');

### Conclusion:
Platforms gross sales differ greatly. Some sell less than 50 millions while other reach a billion total sales.

## Platforms with greatest sales

In [ ]:
(games.groupby('platform')['total_sales'].sum().sort_values(ascending = False))/len(games)*100

In [ ]:
top_18 = games.pivot_table(index = ['platform'], values = ('total_sales'), aggfunc = 'sum').reset_index().sort_values("total_sales", ascending=False)
top_18 = top_18.nlargest(18, 'total_sales', keep = "first")

### Create the only_top_platform dataframe

In [ ]:
top_platform = top_18['platform'].tolist()
only_top_platform = games.query('platform in @top_platform')

### Visualize the greatest sales platform over time

In [ ]:
fig, ax = plt.subplots()
only_top_platform.pivot_table(index = ['platform','year_of_release' ], values = 'total_sales', aggfunc = 'sum').reset_index().plot.scatter(x='year_of_release', y='platform', s='total_sales', figsize = (13,8), ax = ax)
ax.axvline(2004, color="red", linestyle="--", lw=1, label="xx");
#ax.axhline("PS4", color="red", linestyle="--", lw=1, label="xx")

#### Conclusion
Some platforms like PS had the highest sales before the year 2000, just to die and be replaced by its successors, PS2 and PS3.  Others like PC have had a steady, though low, number of sales over time.  On average, platforms seem to have/are designed to have a lifespan.  Moreover, year 2006 is most noteworthy because most of the platforms that exist until 2016 seem to have originated that year.

#### Current platforms

In [ ]:
current_platforms_2016 = only_top_platform[only_top_platform["year_of_release"] == 2016]

In [ ]:
current_platforms_2016.platform.unique()

In [ ]:
platform_list = ['PS4', '3DS', 'XOne', 'PS3', 'PC', 'X360', 'Wii']
current_platforms    = only_top_platform[only_top_platform['platform'].isin(platform_list)]

In [ ]:
games_2006 = current_platforms[current_platforms["year_of_release"] > 2005]
games_2006.platform.unique()

In [ ]:
games_2006['lifecycle']  = current_platforms.groupby(
'platform')['year_of_release'].transform(lambda s: s.max() - s.min())

In [ ]:
games_2006['lifecycle'].describe()

In [ ]:
sns.set(rc={'figure.figsize':(20,7)})
ax = sns.violinplot(x="platform" , y="year_of_release", split=True, width=1, scale="width", palette="Set3",inner="quartile", data=games_2006)
ax.tick_params(axis='x', labelrotation = 90)
plt.show()

#### Conclusion:
Among the most successful but ongoing platforms, the mean age is 12.5 and median age of around 10 years and a min of 3 and a max of 31 years.  How prolific in terms of releases each platform is shown in these violin plots. While X360 and PS3 are the most prolific, PS4, XOne and Wii have variable level of game releases during their lifespan.

#### Dead platforms

In [ ]:
platform_list = ['PS4', '3DS', 'XOne', 'PS3', 'PC', 'X360', 'Wii']
dead_platforms    = only_top_platform[~only_top_platform['platform'].isin(platform_list)]

In [ ]:
dead_platforms['lifecycle']  = dead_platforms.groupby('platform')['year_of_release'].transform(lambda s: s.max() - s.min())
dead_platforms['lifecycle'].describe()

In [ ]:
sns.set(rc={'figure.figsize':(20,7)})
ax = sns.violinplot(x="platform" , y="year_of_release", split=True, width=1, scale="width", palette="Set3",inner="quartile", data=dead_platforms)
ax.tick_params(axis='x', labelrotation = 90)
plt.show()

#### Conclusion:
Of the greatest sales platforms that are decommissioned, the mean lifespan of 13.5 and median lifespan is 11 years and a minimum of 6 and a maximum of 28 years.  What is noteworthy is that the standard deviation is around 6 years compared to a standard deviation of 11 years for those successful platforms that survived until today.  In reality though, our sample is so small that it is swayed by the one platform with the longest longevity. PS2 has had a constantly prolific life until it was replaced by PS3. Other platforms have ebbed and flawed in terms of releases.

#### Conclusion:
If we use the violin for 18 most profitable platforms, we do see that only a few such as PC and DS have a long lifespan.  Among the most profitable and still existing platforms, only PC remains. The 18 platform plot show that since the 1980 we have 2 profitable platforms every couple of years with a life span between a decade and a decade and a half.  

## Current platforms leading sales

In [ ]:
df1 = games_2006.groupby(['year_of_release', 'platform'])['total_sales'].sum().unstack('platform').fillna(0)
df1[['Wii', 'X360', 'PS3', 'PS4', '3DS', 'PC', 'XOne']].plot(kind='bar', stacked=True)
plt.title("Game release distribution by platform")
plt.xlabel('Years')
plt.ylabel('Count');

### Conclusion:
PC is a platform with greatest sales over time.  However, it does not fare well on average. Let's start with the year 2006.

### Calculate trends of sales year-on-year

In [ ]:
games_year = pd.pivot_table(games_2006, index='year_of_release', columns='platform', values='total_sales', aggfunc=sum, fill_value=0)
games_year.head()

In [ ]:
trend = games_year - games_year.shift(+1)
trend.head(5)

In [ ]:
ax = trend.plot.bar(figsize=(15, 5))
patches, labels = ax.get_legend_handles_labels()
ax.legend(patches, labels, loc='lower left')
plt.title("Yearly sales trends by platform")
plt.xlabel('Years')
plt.ylabel('Count');

#### Conclusion: lets see
Sales of all these platforms have plummeted year on year. Since 2010, the platform market has become more volatile than before.  Perhaps it is market saturation or that users now play online games more.  Something is happening.  If we consider the lifespan of a platform and its profitability.  I choose PS4 and XOne, 3DS and PC.  I choose PC due to its longevity.  But the whole market is shrinking.

In [ ]:
total_sale = games_2006.groupby(['platform','year_of_release'])['total_sales'].sum().reset_index()

In [ ]:
total_sale.boxplot(column='total_sales',by='platform', figsize=(13,5));

#### Conclusion:
Global sales difference by platform differs tremedously from one platform to another.

##  Effects of reviews one popular platform

## PS3

In [ ]:
PS3 = games_2006.loc[(games_2006['platform'] == 'PS3')]

In [ ]:
modelPS3 = ols('total_sales ~ critic_score + user_score ', data=PS3)
results = modelPS3.fit()
results.summary()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
sns.regplot(x = "user_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax = ax1,
            data = PS3)
ax1.set(ylim=(0, 1))


sns.regplot(x = "critic_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax=ax2,
            data = PS3)
ax2.set(ylim=(0, 1));

### Conclusion:
PS3 share of reviews like all others are so slim they are insignificant.

In [ ]:
corr = games_2006.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)]= True

ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(10, 220,  as_cmap=True),
    square=True,
    mask=mask,
    annot=True,
    annot_kws={"size": 7}
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

### Conclusion:
Total sales is no correlated with user_score; it is midely positively correlated with critic score.  We also see that total sales is not correlated with ratings either.

### PS4  df

In [ ]:
PS4 = games_2006.loc[(games_2006['platform'] == 'PS4')]

In [ ]:
modelPS4 = ols('total_sales ~ critic_score + user_score ', data=PS4)
results = modelPS4.fit()
results.summary()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
sns.regplot(x = "user_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax = ax1,
            data = PS4)
ax1.set(ylim=(0, 1.2))


sns.regplot(x = "critic_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax=ax2,
            data = PS4)
ax2.set(ylim=(0, 1.2));

### Conclusion:
PS4 share of reviews is mixed: as user score increase, PS4 sale decrease and as critic score increase, its sales increase.  However, the effect are very slim.  It is noteworthy to mention that the confidence interval around the regression line of user score is to wide.

##  XOne

In [ ]:
XOne = games_2006.loc[(games_2006['platform'] == 'XOne')]

In [ ]:
modelXOne = ols('total_sales ~ critic_score + user_score ', data=XOne)
results = modelXOne.fit()
results.summary()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
sns.regplot(x = "user_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax = ax1,
            data = XOne)
ax1.set(ylim=(0, 2))


sns.regplot(x = "critic_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax=ax2,
            data = XOne)
ax2.set(ylim=(0, 2));

### Conclusion:
XOne share of reviews is also mixed: as user score increase, XOne sale decrease and as critic score increase, its sales increase.  However, the effect are very slim.  It is noteworthy to mention that the confidence interval around the regression line of user score is to wide.

## 3DS 

In [ ]:
_3DS = games_2006.loc[(games_2006['platform'] == '3DS')]

In [ ]:
model_3DS = ols('total_sales ~ critic_score + user_score ', data=_3DS)
results = model_3DS.fit()
results.summary()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
sns.regplot(x = "user_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax = ax1,
            data = _3DS)
ax1.set(ylim=(-2, 2))


sns.regplot(x = "critic_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax=ax2,
            data = _3DS)
ax2.set(ylim=(-2, 2));

### Conclusion:
3DS share of reviews move in the same direction: as user and critic scores increase, 3DS sale increase.  Here too the effects are minimal.

## PC

In [ ]:
PC = games_2006.loc[(games_2006['platform'] == 'PC')]

In [ ]:
modelPC = ols('total_sales ~ critic_score + user_score ', data=PC)
results = modelPC.fit()
results.summary()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
sns.regplot(x = "user_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax = ax1,
            data = PC)
ax1.set(ylim=(0, 1))


sns.regplot(x = "critic_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax=ax2,
            data = PC)
ax2.set(ylim=(0, 1));

### Conclusion:
PC share of reviews like all others are so slim they are insignificant.

## Wii

In [ ]:
Wii = games_2006.loc[(games_2006['platform'] == 'Wii')]

In [ ]:
modelWii = ols('total_sales ~ critic_score + user_score ', data=Wii)
results = modelWii.fit()
results.summary()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
sns.regplot(x = "user_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax = ax1,
            data = PC)
ax1.set(ylim=(0, 1))


sns.regplot(x = "critic_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax=ax2,
            data = Wii)
ax2.set(ylim=(0, 1));

### Conclusion:
Wii share of reviews like all others are so slim they are insignificant.

###### X360

In [ ]:
X360 = games_2006.loc[(games_2006['platform'] == 'X360')]

In [ ]:
modelX360 = ols('total_sales ~ critic_score + user_score ', data=X360)
results = modelX360.fit()
results.summary()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
sns.regplot(x = "user_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax = ax1,
            data = PC)
ax1.set(ylim=(0, 1))


sns.regplot(x = "critic_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax=ax2,
            data = X360)
ax2.set(ylim=(0, 1));

### Conclusion:
X360 share of reviews like all others are so slim they are insignificant.

## All Platforms combined

In [ ]:
model = ols('total_sales ~ critic_score + user_score ', data=games_2006)
results = model.fit()
results.summary()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
sns.regplot(x = "user_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax = ax1,
            data = games)
ax1.set(ylim=(0, 1))


sns.regplot(x = "critic_score",
            y = "total_sales", 
             ci=95, 
            scatter_kws={"s": 1}, ax=ax2,
            data = games_2006)
ax2.set(ylim=(0, 1));

### Conclusion:
Sales from all top platforms do not seem to be  affected by reviews, be it user reviews or critic reviews.

## Same games on different platforms working only with platforms and years I found relevant

In [ ]:
games_2006['repeat_games'] = games_2006.groupby('name')['platform'].transform("count")

In [ ]:
games_2006.repeat_games.value_counts()

In [ ]:
same_game = games_2006.loc[games_2006['repeat_games'] > 1]

In [ ]:
same_game.name.nunique()

In [ ]:
same_game['repeats_current_platform'] = same_game.groupby('name')['platform'].transform("count")

In [ ]:
same_game['repeats_current_platform'].value_counts()

In [ ]:
same_game_equal5 = same_game.loc[
    same_game['repeats_current_platform'] == 5]
same_game_equal5.name.nunique()

In [ ]:
same_game_equal6 = same_game.loc[
    same_game['repeats_current_platform'] == 6]
same_game_equal6.name.nunique()

In [ ]:
sns.barplot(data=same_game_equal6, x='platform', y='total_sales', hue='name');

In [ ]:
same_game_equal7 = same_game.loc[
    same_game['repeats_current_platform'] == 7]
same_game_equal7.name.nunique()

In [ ]:
sns.barplot(data=same_game_equal7, x='platform', y='total_sales', hue='name');

#### Conclusion:
??

### Select top games for another way of looking at the same question

In [ ]:
top_10_repeat_games = same_game.groupby(['name','platform'])['total_sales'].sum().sort_values(ascending = False).head(10)
top_10_repeat_games

In [ ]:
top_10_repeat_games = same_game.nlargest(18, 'total_sales')
top_10_repeat_games.name.unique()

In [ ]:
df = top_10_repeat_games.groupby(['name', 'platform'])['total_sales'].sum().unstack('name').fillna(0)
df[['Grand Theft Auto V', 'Call of Duty: Modern Warfare 3',
       'Call of Duty: Black Ops 3', 'Call of Duty: Black Ops',
       'Call of Duty: Black Ops II', 'Call of Duty: Modern Warfare 2',
       'Grand Theft Auto IV', 'Call of Duty: Ghosts', 'Just Dance 3',
       'Call of Duty 4: Modern Warfare']].plot(kind='bar', stacked=True)
patches, labels = ax.get_legend_handles_labels()
ax.legend(patches, labels, loc ="best");

### Conclusion:
At \\$100 million X360 has the top 10 highest share of the greatest games sold on similar platforms, followed by PS3.

In [ ]:
top_25_repeat_games = same_game.groupby(['name','platform'])['total_sales'].sum().sort_values(ascending = False).head(25)
top_25_repeat_games

In [ ]:
top_25_repeat_games = same_game.nlargest(35, 'total_sales', keep = "first")
top_25_repeat_games.name.unique()

In [ ]:
df1 = top_25_repeat_games.groupby(['name', 'platform'])['total_sales'].sum().unstack('name').fillna(0)
df1[['Grand Theft Auto V', 'Call of Duty: Modern Warfare 3',
       'Call of Duty: Black Ops 3', 'Call of Duty: Black Ops',
       'Call of Duty: Black Ops II', 'Call of Duty: Modern Warfare 2',
       'Grand Theft Auto IV', 'Call of Duty: Ghosts', 'Just Dance 3',
       'Call of Duty 4: Modern Warfare', 'Minecraft',
       'The Elder Scrolls V: Skyrim', 'FIFA 16', 'FIFA Soccer 13',
       'The Sims 3', 'Star Wars Battlefront (2015)',
       'Call of Duty: Advanced Warfare', 'FIFA 17',
       'Call of Duty: World at War', 'Battlefield 3', 'Fallout 4',
       'Just Dance 4', 'Zumba Fitness', 'FIFA 12']].plot(kind='bar', stacked=True)
patches, labels = ax.get_legend_handles_labels()
ax.legend(patches, labels, loc ="best");

#### Conclusion:
At around 135 million, X360 maintains its leadership of the sale of the top 25 most popular games shared on major platforms and followed by PS3 at 120 million.  

In [ ]:
top_50_repeat_games = same_game.groupby(['name','platform'])['total_sales'].sum().sort_values(ascending = False).head(50)

### Conclusion: compare the sales of the same games on other platforms.
Of the top 50 most popular games shared on major platforms, PS3 overtakes X360 as becomes the most popular platform of the top 50 most popular games.

## Most profitable genre

In [ ]:
games_2006.genre.value_counts()/len(games_2006)*100

In [ ]:
top5_popular_genre = games_2006.pivot_table(index=[ 'genre'],values=['total_sales' ], aggfunc=sum).reset_index()\
 .sort_values(by='total_sales', ascending=False).head(5)
top5_popular_genre.head()

In [ ]:
top_5 = games_2006.pivot_table(index = ['genre'], values = ('total_sales'), aggfunc = 'sum').reset_index().sort_values("total_sales", ascending=False)
top_5 = top_5.nlargest(5, 'total_sales', keep = "first")
top_5.genre.unique()

In [ ]:
top_5_genre_games = games_2006.query(' genre in ("Action", "Shooter", "Sports", "Misc", "Role-Playing")')

### Conclusion

In [ ]:
top10_popular_genre_game_pivot = top_5_genre_games.pivot_table(index=[ 'genre', 'name'],
                                          values=['total_sales' ],
                                          aggfunc=sum).reset_index()\
.sort_values(by='total_sales', ascending=False).head(10)
top10_popular_genre_game_pivot.head()

In [ ]:
df_2= top10_popular_genre_game_pivot.groupby(['genre', "name"])['total_sales'].sum().unstack('name').fillna(0)
df_2.plot(kind='bar', figsize=(13, 5), stacked=True);

In [ ]:
top25_popular_genre_game_pivot = top_5_genre_games.pivot_table(index=[ 'genre', 'name'],
                                          values=['total_sales' ],
                                          aggfunc=sum).reset_index()\
.sort_values(by='total_sales', ascending=False).head(25)
top25_popular_genre_game_pivot.head()

In [ ]:
df_2= top25_popular_genre_game_pivot.groupby(['genre', "name"])['total_sales'].sum().unstack('name').fillna(0)
df_2.plot(kind='bar', figsize=(13, 5), stacked=True);

In [ ]:
genre_list      = ['Action', 'Sports', 'Shooter', 'Role-Playing', 'Platform']
greatest_genre    = games_2006[games_2006['genre'].isin(genre_list)]

In [ ]:
df_2 = greatest_genre.groupby(['genre', "platform"])['total_sales'].sum().unstack('genre').fillna(0)
df_2.plot(kind='bar', figsize=(13, 5), stacked=True);

In [ ]:
not_so_great_genre    = games_2006[~games_2006['genre'].isin(genre_list)]

In [ ]:
df_2 = not_so_great_genre.groupby(['genre', "platform"])['total_sales'].sum().unstack('genre').fillna(0)
df_2.plot(kind='bar', figsize=(13, 5), stacked=True);

### Conclusion: 
Shooter is the genre with the top 10 total sales of games which one can assume is profitable though we do not have enough information. When we look at the top 25 sales of games, we see that shooter is closely followed by sports. The top 50 sales of games the same trend of popularity is exhibited.  The trend from the greatest genre in the most profitable platform is also interesting.  PS3 and X360 lead the sales, followed by Wii and PS4.  Games of not so great genres still follow the same trend.  In other words, the great platforms sell both great and not so great genre and titles.

## Regions

### North America

In [ ]:
platform_na = games_2006.pivot_table(index=['platform'],
                                      values =['na_sales'],
                                      aggfunc=sum)\
.reset_index().sort_values(by='na_sales', ascending=False).head(5)
platform_na

In [ ]:
genre_na = games_2006.pivot_table(index=['genre'],
                                      values =['na_sales'],
                                      aggfunc=sum)\
.reset_index().sort_values(by='na_sales', ascending=False).head(5)
genre_na

In [ ]:
fig = plt.figure()
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)
ax1=genre_na.groupby(['genre']).sum().plot(kind='pie', y='na_sales',  ax=ax1)
ax1.axis('equal')
ax2=platform_na.groupby(['platform']).sum().plot(kind='pie', y='na_sales',  ax=ax2)
ax2.axis('equal');

#### Conclusion:
In the North American market, X360 is the most popular platform followed by Wii and PS3 and the most popular genre are action, sports and shooter.

### Europe

In [ ]:
platform_eu = games_2006.pivot_table(index=['platform'],
                                      values =['eu_sales'],
                                      aggfunc=sum)\
.reset_index().sort_values(by='eu_sales', ascending=False).head(5)
platform_eu

In [ ]:
genre_eu = games_2006.pivot_table(index=['genre'],
                                      values =['eu_sales'],
                                      aggfunc=sum)\
.reset_index().sort_values(by='eu_sales', ascending=False).head(5)
genre_eu

In [ ]:
fig = plt.figure()
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)
ax1=genre_eu.groupby(['genre']).sum().plot(kind='pie', y='eu_sales',  ax=ax1)
ax1.axis('equal')
ax2=platform_eu.groupby(['platform']).sum().plot(kind='pie', y='eu_sales',  ax=ax2)
ax2.axis('equal');

#### Conclusion:
In the European market again, X360 is the most popular platform followed by Wii and PS3 and the most popular genre are action, sports and shooter.

### Japan

In [ ]:
platform_jp = games_2006.pivot_table(index=['platform'],
                                      values =['jp_sales'],
                                      aggfunc=sum)\
.reset_index().sort_values(by='jp_sales', ascending=False).head(5)
platform_jp

In [ ]:
genre_jp = games_2006.pivot_table(index=['genre'],
                                      values =['jp_sales'],
                                      aggfunc=sum)\
.reset_index().sort_values(by='jp_sales', ascending=False).head(5)
genre_jp

In [ ]:
fig = plt.figure()
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)
ax1=genre_jp.groupby(['genre']).sum().plot(kind='pie', y='jp_sales',  ax=ax1)
ax1.axis('equal')
ax2=platform_jp.groupby(['platform']).sum().plot(kind='pie', y='jp_sales',  ax=ax2)
ax2.axis('equal');

#### Conclusion:
In the Japanese market, 3DS is the most popular platform followed by Wii and PS3 and the most popular genre are actionand role-play.

### Other regions

In [ ]:
platform_other = games_2006.pivot_table(index=['platform'],
                                      values =['other_sales'],
                                      aggfunc=sum)\
.reset_index().sort_values(by='other_sales', ascending=False).head(5)
platform_other

In [ ]:
genre_other = games_2006.pivot_table(index=['genre'],
                                      values =['other_sales'],
                                      aggfunc=sum)\
.reset_index().sort_values(by='other_sales', ascending=False).head(5)
genre_other

In [ ]:
fig = plt.figure()
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)
ax1=genre_other.groupby(['genre']).sum().plot(kind='pie', y='other_sales',  ax=ax1)
ax1.axis('equal')
ax2=platform_other.groupby(['platform']).sum().plot(kind='pie', y='other_sales',  ax=ax2)
ax2.axis('equal');

#### Conclusion:
In the rest of the world market, PS3 is the most popular platform followed by X360 and Wii and the most popular genre are action, sports and shooter.

### Compare genre by region

In [ ]:
fig = plt.figure()
ax1 = plt.subplot(141)
ax2 = plt.subplot(142)
ax3 = plt.subplot(143)
ax4 = plt.subplot(144)
ax1=genre_na.groupby(['genre']).sum().plot(kind='pie', y='na_sales',  ax=ax1)
ax1.axis('equal')
ax2=genre_eu.groupby(['genre']).sum().plot(kind='pie', y='eu_sales',  ax=ax2)
ax2.axis('equal')
ax3=genre_jp.groupby(['genre']).sum().plot(kind='pie', y='jp_sales',  ax=ax3)
ax3.axis('equal')
ax4=genre_other.groupby(['genre']).sum().plot(kind='pie', y='other_sales',  ax=ax4)
ax4.axis('equal');

In [ ]:
grouped_genre = greatest_genre.pivot_table(index=['genre'],
                                      values =['na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'total_sales'],
                                      aggfunc=sum)\
.reset_index()\
.sort_values(by='total_sales', ascending=False)

grouped_genre

In [ ]:
grouped_genre.plot(x="genre", y=[
    'na_sales', 'eu_sales', 'jp_sales', 'other_sales'], kind="bar");

#### Conclusion:
The share of the north american market is the largest in the world in all genres followed by that of Europe.  Action is by far the most popular genre in in the world.

## Compare platforms per region

In [ ]:
fig = plt.figure()
ax1 = plt.subplot(141)
ax2 = plt.subplot(142)
ax3 = plt.subplot(143)
ax4 = plt.subplot(144)
ax1=platform_na.groupby(['platform']).sum().plot(kind='pie', y='na_sales',  ax=ax1)
ax1.axis('equal')
ax2=platform_eu.groupby(['platform']).sum().plot(kind='pie', y='eu_sales',  ax=ax2)
ax2.axis('equal')
ax3=platform_jp.groupby(['platform']).sum().plot(kind='pie', y='jp_sales',  ax=ax3)
ax3.axis('equal')
ax4=platform_other.groupby(['platform']).sum().plot(kind='pie', y='other_sales',  ax=ax4)
ax4.axis('equal');

In [ ]:
grouped_platform = games_2006.pivot_table(index=['platform'],
                                      values =['na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'total_sales'],
                                      aggfunc=sum)\
.reset_index()\
.sort_values(by='total_sales', ascending=False)

In [ ]:
grouped_platform.plot(x="platform", y=[
    'na_sales', 'eu_sales', 'jp_sales', 'other_sales'], kind="bar");

### Conclusion
The north American market of platform sales is the the highest in the world followed by that of Europe.  The share of PS3 and PS4 were the highest in the European region. 

## Effect of ESRB

In [ ]:
rating_region = games_2006.pivot_table(index=['rating'],
                                      values =['na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'total_sales'],
                                      aggfunc=sum)\
.reset_index()\
.sort_values(by='total_sales', ascending=False)

In [ ]:

corr = rating_region.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)]= True

ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(10, 220,  as_cmap=True),
    square=True,
    mask=mask,
    annot=True,
    annot_kws={"size": 7}
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

### Conclusion:
It does appear that ESRB rating with 50\% na is more correlated with sales in the regions.

In [ ]:
rating_region.plot(x="rating", y=['na_sales', 'eu_sales', 'jp_sales', 'other_sales'], kind="bar");

#### Conclusion:
North american sales of rating M, E, T and E10+ are the highest followed European sales.

### Using rating of ESRB encoded

In [ ]:
rating_region_mode = games_2006.pivot_table(index=['rating_na_filled_mode'],
                                      values =['na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'total_sales'],
                                      aggfunc=sum)\
.reset_index()\
.sort_values(by='total_sales', ascending=False)

rating_region_mode

In [ ]:

plt.figure(figsize=(16, 6))


corr = rating_region_mode.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)]= True

ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(10, 220,  as_cmap=True),
    square=True,
    mask=mask,
    annot=True,
    annot_kws={"size": 7}
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);



### Conclusion
The ESRB rating that was filled with the mode is moderately correlated with sales in all regions

In [ ]:
rating_region_mode.plot(x="rating_na_filled_mode", y=[
    'na_sales', 'eu_sales', 'jp_sales', 'other_sales'], kind="bar");

#### Conclusion:
North American sales of E, T, M, E10+ are the highest, followed by Europe.

# Hypothesis testing

## Platform

### Check the normality of the user_score variable for both platforms

In [ ]:
#import pingouin as pg
pg.normality(XOne.user_score, method='normaltest')

In [ ]:
pg.normality(PC.user_score, method='normaltest')

#### Conclusion
Since the variables are not normally distributed, I choose the levene test.  The null hypothesis is that the two samples originate from the same distribution and hence are similar.  The alternative is that they are different.  I choose the commonly used level of significance of alpha is 0.05.  That is the degree of error I accept is as low s 5\%.

In [ ]:
p_value_levene=st.levene(XOne.user_score, PC.user_score).pvalue.astype(int)
if p_value_levene<0.05:
    print('Reject H0: variance of sample 1 is not equal to variance of sample 2')
else:
    print( "Fail to Reject H0: We don't have enough evidence to say that variances of sample 1 \
    and sample 2 are not the same") 

I also do a ttest just for fun to compare.

In [ ]:
t, pval = ttest_ind(XOne.user_score, PC.user_score)

print("p-values" + str(pval))
if pval < 0.05:    # alpha value is 0.05 or 5%
    print("We are rejecting the null hypothesis")
else:
    print("We are not rejecting the null hypothesis")

## Genre

In [ ]:
genre_action = games_2006.query('genre == "Action"')

In [ ]:
genre_sports = games_2006.query('genre == "Sports"')

In [ ]:
pg.normality(genre_action.user_score, method='normaltest')

In [ ]:
pg.normality(genre_sports.user_score, method='normaltest')

#### Conclusion
Since the variables are not normally distributed, I choose the levene test.  The null hypothesis is that the two samples originate from the same distribution and hence are similar.  The alternative is that they are different.  I choose the commonly used level of significance of alpha is 0.05.  That is the degree of error I accept is as low s 5\%.

In [ ]:
p_value_levene=st.levene(genre_action.user_score, genre_sports.user_score).pvalue.astype(int)
if p_value_levene<0.05:
    print('Reject H0: variance of sample 1 is not equal to variance of sample 2')
else:
    print( "Fail to Reject H0: We don't have enough evidence to say that variances of sample 1 \
    and sample 2 are not the same") 

In [ ]:
t, pval = ttest_ind(genre_action.user_score, genre_sports.user_score)

print("p-values" + str(pval))
if pval < 0.05:    # alpha value is 0.05 or 5%
    print("We are rejecting the null hypothesis")
else:
    print("We are not rejecting the null hypothesis")

# General Conclusion:
- Technology is as such that the lifespan of a platform is limited.  Of those platforms with greatest sales, the lifespan of current platform is longer than those who have been decommissioned.
- The market for the platforms has been exponentially shrinking.
- Games are released on several platforms at once.  
- A  successful platform carries multiple genre at once.  Some are more successful than others.
- There does not appear to be a genre niche market among the most successful platforms.  They all offer several genres.
- Rating, be it user or critic rating, does not appear to have a great influence on sales.
- ESRB rating does appear to be correlated with sales in every region.  However, because of the amount of missing values, this finding is questionable although a similar and less strong correlation was found with the missing values were repalced by the mode.
- The North American market is the strongest in the world for platform sales.  
- The most popular genre of games in the world is action.
- Depending on whether we fill the nas in the ESRB rating or not, the most popular rating is either in this order (E,T,M) or in that order (M,E,T)
- The user rating of the two platforms XOne and PC differ
- The user rating of action and sports gener also differ.